In [1]:
import numpy as np
from jesse import helpers, research

_, candles = research.get_candles(
    "Binance Perpetual Futures",
    "BTC-USDT",
    "1m",
    helpers.date_to_timestamp("2020-01-01"),
    helpers.date_to_timestamp("2025-07-01"),
    warmup_candles_num=0,
    caching=False,
    is_for_jesse=False,
)

print(candles.shape)

np.save("data/btc_1m.npy", candles)

2025-07-06 13:10:27,331	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


(2891520, 6)


In [1]:
import numpy as np

candles = np.load("data/btc_1m.npy")
candles = candles[candles[:, 5] > 0]
print(candles.shape)

(2891355, 6)


In [5]:
DATA_PATH = "/Users/yangqiuyu/Github/jesse-trade/data/btc_1m.npy"

with open("data/pysr_loss.jl", "r") as f:
    custom_julia_loss = f.read().replace("btc_1m.npy", DATA_PATH)

print(custom_julia_loss)

function kurtosis_loss(tree, dataset::Dataset{T,L}, options) where {T,L}
    using NPZ
    using Statistics

    candles = npzread("/Users/yangqiuyu/Github/jesse-trade/data/btc_1m.npy")
    candles = candles[candles[:, 6].>0, :]

    # build bar function
    function build_bar_by_cumsum(candles, condition, threshold)
        n = size(candles, 1)
        bars = zeros(Float64, n, 6)
        bar_index = 1

        # 初始化第一个bar
        bar_timestamp = candles[1, 1]
        bar_open = candles[1, 2]
        bar_close = candles[1, 3]
        bar_high = candles[1, 4]
        bar_low = candles[1, 5]
        bar_volume = candles[1, 6]
        bar_cumsum = condition[1]

        for i in 2:n
            if bar_cumsum <= threshold
                bar_cumsum += condition[i]
                bar_timestamp = max(bar_timestamp, candles[i, 1])
                bar_volume += candles[i, 6]
                bar_high = max(bar_high, candles[i, 4])
                bar_low = min(bar_low, candles[i, 5])
          